In [1]:
# Flask = Spring
from flask import Flask
app = Flask(__name__)

# Spring Controller와 동일
@app.route('/')
def hello():
    return '<h1>Hello Python</h1>'

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [1]:
import requests

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "",
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)


Error!


In [6]:
import json
import requests
import datetime
import os

# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = 'res/kakao_msg/kakao_token.json'

# 저장
def save_tokens(filename, tokens):
    with open(filename, 'w') as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# 갱신
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : "",
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)

    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_filename = filename + '.' + now
        os.rename(filename, backup_filename)
        
        # 업데이트
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [9]:
# Kakao Talk 메세지 보내기
import json
import requests

# 토큰 불러오기
tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

# 요청 url
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# request parameter
headers = {
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "카카오톡 메세지 테스트!!",
        "link" : {
            # 이동할 링크
            "web_url" : "www.naver.com"
        }
    })
}

# 메세지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print('Error!')
else:
    tokens = response.json()
    print('카톡 메세지 보내기 성공')

200
카톡 메세지 보내기 성공


In [ ]:
from flask import Flask, url_for, request
app = Flask(__name__)

@app.route('/')
def hello():
    return '<h1>Hello Python</h1>'

# <> 가변인자
@app.route('/member/<id>')
def get_member(id):
    return 'member: ' + id

@app.route('/test', methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        id = request.form['id']
        email = request.form['email']
        
        if not id and not email:
            return add_entity(request.form)
    else: 
        err = 'Invalid'
        
    return err
        
if __name__ == '__main__':
    #with app.test_request_context():
        # hello(), get_member()의 url 확인
        #print (url_for('hello'))
        #print (url_for('get_member', id='test'))
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:20:21] "GET / HTTP/1.1" 200 -


In [4]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app)

@app.route('/login', methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        print('POST 처리')
        #id = request.form['id']
        #pw = request.form['pw']
        # request.get_json(): 상대편에서 보낸 json 내용을 읽음
        data = request.get_json()
        
        return jsonify(data)
    
    else: 
        print('GET 처리')
        err = 'Invalid'
        
    return err
        
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 20:31:50] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 20:31:51] "POST /login HTTP/1.1" 200 -


POST 처리
